In [ ]:
!pip install spacy pandas scikit-learn PyPDF2
!python -m spacy download en_core_web_sm


In [ ]:
python app.py

In [ ]:
import os

# Create folders
os.makedirs("templates", exist_ok=True)
os.makedirs("uploads", exist_ok=True)

print("✅ Project structure readyimport os
from flask import Flask, render_template, request, send_file
import PyPDF2
import spacy
import pandas as pd
from werkzeug.utils import secure_filename
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text = ''
        for page in reader.pages:
            text += page.extract_text() or ''
    return text

def preprocess(text):
    doc = nlp(text.lower())
    return ' '.join([token.lemma_ for token in doc if token.is_alpha and not token.is_stop])

@app.route("/", methods=["GET", "POST"])
def index():
    roles = {
        "Python Developer": "Looking for a Python Developer skilled in Flask, Django, APIs, and data handling with Pandas and NumPy.",
        "Data Scientist": "Seeking a Data Scientist proficient in statistics, machine learning, Python, and visualization tools like Matplotlib and Seaborn.",
        "Web Developer": "Hiring a Web Developer experienced in HTML, CSS, JavaScript, React, and backend integration with Flask or Node.js.",
        "AI/ML Engineer": "Searching for an AI/ML Engineer with strong knowledge of deep learning, TensorFlow, PyTorch, and NLP tools like SpaCy and transformers."
    }

    if request.method == "POST":
        jd = request.form["jd"]
        files = request.files.getlist("resumes")

        if not jd or not files:
            return render_template("index.html", error="Please provide a job description and upload resumes.", roles=roles)

        jd_clean = preprocess(jd)
        resume_texts, resume_names = [], []

        for file in files:
            filename = secure_filename(file.filename)
            path = os.path.join(UPLOAD_FOLDER, filename)
            file.save(path)
            text = extract_text_from_pdf(path)
            cleaned = preprocess(text)
            resume_texts.append(cleaned)
            resume_names.append(filename)

        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([jd_clean] + resume_texts)
        scores = cosine_similarity(vectors[0:1], vectors[1:]).flatten()

        df = pd.DataFrame({
            'Resume': resume_names,
            'Score': scores
        }).sort_values(by='Score', ascending=False).reset_index(drop=True)

        df.to_csv("ranked_resumes.csv", index=False)
        return render_template("index.html", roles=roles, tables=[df.to_html(classes="table", index=False)], success=True)

    return render_template("index.html", roles=roles)

@app.route("/download")
def download():
    return send_file("ranked_resumes.csv", as_attachment=True)

if __name__ == "__main__":
    app.run(debug=True)



In [ ]:
<!DOCTYPE html>
<html>
<head>
  <title>AI Resume Ranker</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
  <div class="container">
    <h1>🤖 AI Resume Ranker</h1>

    {% if error %}
      <div class="alert">{{ error }}</div><!DOCTYPE html>
<html>
<head>
  <title>AI Resume Ranker</title>
  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
  <div class="container">
    <h1>🤖 AI Resume Ranker</h1>

    {% if error %}
      <div class="alert">{{ error }}</div>
    {% endif %}

    <form method="POST" enctype="multipart/form-data">
      <label for="role">Choose a job role:</label>
      <select id="role" onchange="fillJD(this)">
        <option disabled selected>Select a template</option>
        {% for role in roles %}
          <option value="{{ roles[role] }}">{{ role }}</option>
        {% endfor %}
      </select>

      <label>📄 Job Description:</label>
      <textarea name="jd" id="jd" rows="6" required placeholder="Paste or select a job description template"></textarea>

      <label>📂 Upload PDF Resumes:</label>
      <input type="file" name="resumes" accept=".pdf" multiple required>

      <button type="submit">🚀 Rank Resumes</button>
    </form>

    {% if success %}
      <div class="results">
        <h2>✅ Ranked Resumes</h2>
        {{ tables|safe }}
        <a href="/download" class="download-btn">📥 Download CSV</a>
      </div>
    {% endif %}
  </div>

  <script>
    function fillJD(select) {
      document.getElementById("jd").value = select.value;
    }
  </script>
</body>
</html>

    {% endif %}

    <form method="POST" enctype="multipart/form-data">
      <label for="role">Choose a job role:</label>
      <select id="role" onchange="fillJD(this)">
        <option disabled selected>Select a template</option>
        {% for role in roles %}
          <option value="{{ roles[role] }}">{{ role }}</option>
        {% endfor %}
      </select>

      <label>📄 Job Description:</label>
      <textarea name="jd" id="jd" rows="6" required placeholder="Paste or select a job description template"></textarea>

      <label>📂 Upload PDF Resumes:</label>
      <input type="file" name="resumes" accept=".pdf" multiple required>

      <button type="submit">🚀 Rank Resumes</button>
    </form>

    {% if success %}
      <div class="results">
        <h2>✅ Ranked Resumes</h2>
        {{ tables|safe }}
        <a href="/download" class="download-btn">📥 Download CSV</a>
      </div>
    {% endif %}
  </div>

  <script>
    function fillJD(select) {
      document.getElementById("jd").value = select.value;
    }
  </script>
</body>
</html>


In [ ]:
body {
  font-family: 'Segoe UI', sans-serif;
  background: #f0f2f5;
  padding: 30px;
}

.container {
  max-width: 800px;
  margin: auto;
  background: white;
  padding: 30px;
  border-radius: 12px;
  box-shadow: 0 0 10px rgba(0,0,0,0.1);
}

h1 {
  text-align: center;
  color: #333;
}

label {
  font-weight: bold;
  display: block;
  margin-top: 20px;
}

textarea, input[type="file"], select {
  width: 100%;
  padding: 10px;
  margin-top: 6px;
  border: 1px solid #ccc;
  border-radius: 8px;
}

button {
  background-color: #007bff;
  color: white;
  padding: 12px;
  border: none;
  margin-top: 20px;
  border-radius: 8px;
  width: 100%;
  font-size: 16px;
  cursor: pointer;
}

button:hover {
  background-color: #0056b3;
}

.alert {
  color: white;
  background: #ff4444;
  padding: 10px;
  margin-bottom: 20px;
  border-radius: 8px;
  text-align: center;
}

.table {
  width: 100%;
  border-collapse: collapse;
  margin-top: 20px;
}

.table th, .table td {
  border: 1px solid #ddd;
  padding: 10px;
  text-align: left;
}

.table th {
  background: #f2f2f2;
}

.download-btn {
  display: inline-block;
  margin-top: 15px;
  background-color: #28a745;
  color: white;
  padding: 10px 15px;
  border-radius: 8px;
  text-decoration: none;
}

.download-btn:hover {
  background-color: #1e7e34;
}
